In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data63881


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries
!pip install lightgbm==2.3.1 -t /home/aistudio/external-libraries
# !pip install catboost==0.23 -t /home/aistudio/external-libraries
# !pip install jieba -t /home/aistudio/external-libraries
# !pip install gensim -t /home/aistudio/external-libraries
!pip uninstall --yes pandas
!pip install pandas==1.0.5 -t /home/aistudio/external-libraries

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.2MB 14.5MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 14.4MB/s eta 0:00:01
     |████████████████████████████████| 25.9MB 11.0MB/s eta 0:00:01
     |████████████████████████████████| 14.5MB 8.6MB/s eta 0:00:011
     |████████████████████████████████| 307kB 18.6MB/s eta 0:00:01


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [ ]:
train_transaction = pd.read_csv('data/data63881/train_transaction.csv')
test_transaction = pd.read_csv('data/data63881/test_transaction.csv')
train_identity = pd.read_csv('data/data63881/train_identity.csv')
test_identity = pd.read_csv('data/data63881/test_identity.csv')

In [ ]:
train = train_transaction.merge(train_identity, on='TransactionID', how='left')
test = test_transaction.merge(test_identity, on='TransactionID', how='left')
data = pd.concat([train, test], axis=0, ignore_index=True)
del train, test
gc.collect()

In [ ]:
object_cols = ['ProductCD', 'card4', 'card6', 'DeviceType', 'DeviceInfo', 'P_emaildomain', 'R_emaildomain']
M_cols = ['M{}'.format(i) for i in range(1, 10)]
id_cols = ['id_12', 'id_16', 'id_27', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37', 'id_38', 'id_15',
           'id_23', 'id_34', 'id_30', 'id_31', 'id_33']
cat_cols = object_cols + M_cols + id_cols

In [ ]:
for i in cat_cols:
    le = LabelEncoder()
    data[i] = le.fit_transform(data[i].astype(str))

In [ ]:
train = data[data['isFraud'].notnull()]
test = data[data['isFraud'].isnull()]

y = train['isFraud']
train.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
test.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)
used_cols = train.columns
test = test[used_cols]

X_train, X_valid, y_train, y_valid = train_test_split(train, y, random_state=2020)

In [ ]:
def auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52):
    useful_dict = dict()
    useless_dict = dict()
    params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'metric': 'auc',
        'learning_rate': 0.1,
        'num_leaves': 31,
        'lambda_l1': 0,
        'lambda_l2': 1,
        'num_threads': 23,
        'min_data_in_leaf': 20,
        'first_metric_only': True,
        'is_unbalance': True,
        'max_depth': -1,
        'seed': 2020
    }
    for i in cols:
        print(i)
        try:
            lgb_train = lgb.Dataset(X_train[[i]].values, y_train)
            lgb_valid = lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
            lgb_model = lgb.train(
                params,
                lgb_train,
                valid_sets=[lgb_valid, lgb_train],
                num_boost_round=1000,
                early_stopping_rounds=50,
                verbose_eval=500
            )
            print('*' * 10)
            print(lgb_model.best_score['valid_0']['auc'])
            if lgb_model.best_score['valid_0']['auc'] > threshold:
                useful_dict[i] = lgb_model.best_score['valid_0']['auc']
            else:
                useless_dict[i] = lgb_model.best_score['valid_0']['auc']
        except:
            print('Error: ', i)
    useful_cols = list(useful_dict.keys())
    useless_cols = list(useless_dict.keys())
    return useful_dict, useless_dict, useful_cols, useless_cols


useful_dict, useless_dict, useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, used_cols, threshold=0.52)

In [ ]:
useless_cols

In [ ]:
X_train = X_train[useful_cols]
X_valid = X_valid[useful_cols]
test = test[useful_cols]

In [ ]:
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)
params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
    # 'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}
valid_model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    num_boost_round=1000000,
    early_stopping_rounds=200,
    verbose_eval=300
)
pred = valid_model.predict(test)
sub = pd.DataFrame({'id': range(len(test))})
sub['isFraud'] = pred
sub.to_csv('./sub/basline.csv', index=False, header=None)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 